In [3]:
import pandas as pd
import numpy as np
import requests
import io
from datetime import datetime

## Pruebas de carga local

## Pruebas de carga desde una URL

## Inicio

In [4]:
url_items = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat"
url_ratin = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat"
url_users = "https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/users.dat"

itm_file = pd.read_csv(url_items, sep='::',encoding='utf8',error_bad_lines=False,engine='python')
use_file = pd.read_csv(url_users, sep='::',encoding='utf8',error_bad_lines=False,engine='python')
rat_file = pd.read_csv(url_ratin, sep='::',encoding='utf8',error_bad_lines=False,engine='python')

itm_file.columns = ['movie_id', 'movie_title', 'genre']
use_file.columns = ['userid', 'twitter_id']
rat_file.columns = ['user_id','movie_id','rating','rating_timestamp']

In [5]:
itm_file.head() 

,movie_id,movie_title,genre
0,10,La sortie des usines Lumière (1895),Documentary|Short
1,12,The Arrival of a Train (1896),Documentary|Short
2,25,The Oxford and Cambridge University Boat Race ...,NaN
3,91,Le manoir du diable (1896),Short|Horror
4,131,Une nuit terrible (1896),Short|Comedy|Horror


In [6]:
itm_file.shape

(34429, 3)

In [7]:
use_file.head()

,userid,twitter_id
0,2,40501255
1,3,417333257
2,4,138805259
3,5,2452094989
4,6,391774225


In [8]:
use_file.shape

(60281, 2)

In [9]:
use_file = use_file.rename(columns = {'userid':'user_id'})

In [10]:
rat_file.tail()

,user_id,movie_id,rating,rating_timestamp
814353,60280,8695030,1,1567816456
814354,60280,9398640,1,1569026144
814355,60281,816711,8,1371972851
814356,60282,1559547,2,1373287369
814357,60282,2415464,2,1373772560


In [11]:
rat_file.shape

(814358, 4)

In [12]:
rat_file['rating_timestamp'] = pd.to_datetime(rat_file['rating_timestamp'], unit='s')
rat_file.tail()

,user_id,movie_id,rating,rating_timestamp
814353,60280,8695030,1,2019-09-07 00:34:16
814354,60280,9398640,1,2019-09-21 00:35:44
814355,60281,816711,8,2013-06-23 07:34:11
814356,60282,1559547,2,2013-07-08 12:42:49
814357,60282,2415464,2,2013-07-14 03:29:20


In [13]:
rat_file = rat_file.rename(columns = {'rating_timestamp':'date'})
rat_file['time'] = rat_file['date'].astype(str)
rat_file['date'] = rat_file['date'].astype(str)

In [14]:
rat_file.time.dtype

dtype('O')

In [15]:
rat_file['date'] = rat_file['date'].str.split(' ').str[0]
rat_file['time'] = rat_file['time'].str[-8:]

In [16]:
rat_file.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [17]:
itm_file['movie_year'] = itm_file['movie_title']

In [18]:
itm_file['movie_title'] = itm_file['movie_title'].str.split('(').str[0]
itm_file['movie_title'] = itm_file['movie_title'].str.strip()
itm_file['movie_year'] = itm_file['movie_year'].str[-6:]

In [19]:
itm_file['movie_year'] = itm_file['movie_year'].str.split('(').str[1]
itm_file['movie_year'] = itm_file['movie_year'].str.split(')').str[0]

In [20]:
itm_file.head() 

,movie_id,movie_title,genre,movie_year
0,10,La sortie des usines Lumière,Documentary|Short,1895
1,12,The Arrival of a Train,Documentary|Short,1896
2,25,The Oxford and Cambridge University Boat Race,NaN,1895
3,91,Le manoir du diable,Short|Horror,1896
4,131,Une nuit terrible,Short|Comedy|Horror,1896


In [21]:
itm_file.shape

(34429, 4)

In [22]:
df = rat_file

In [23]:
df.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [24]:
df['date']=df['date'].astype(str)

In [25]:
df.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [26]:
df['date']= pd.to_datetime(df['date']) 

In [27]:
df['date'].unique()

array(['2013-07-10T00:00:00.000000000', '2013-07-12T00:00:00.000000000',
       '2013-07-24T00:00:00.000000000', ...,
       '2014-12-11T00:00:00.000000000', '2015-09-25T00:00:00.000000000',
       '2015-09-30T00:00:00.000000000'], dtype='datetime64[ns]')

In [28]:
df.head()

,user_id,movie_id,rating,date,time
0,1,117060,7,2013-07-10,00:13:51
1,1,120755,6,2013-07-10,02:46:00
2,1,317919,6,2013-07-10,22:36:03
3,1,454876,10,2013-07-12,09:25:25
4,1,790724,8,2013-07-24,04:48:40


In [29]:
df = df.sort_values(by='date')
df.reset_index(drop=True, inplace=True)
df.head()

,user_id,movie_id,rating,date,time
0,29235,1772341,8,2013-02-28,23:46:36
1,7115,1411238,6,2013-02-28,14:47:18
2,17750,1454029,8,2013-02-28,21:12:36
3,36365,1922777,6,2013-02-28,18:03:03
4,46678,1333125,3,2013-02-28,17:36:10


In [30]:
df_2 = pd.merge(df,itm_file, on='movie_id')

In [31]:
df_2.head()

,user_id,movie_id,rating,date,time,movie_title,genre,movie_year
0,29235,1772341,8,2013-02-28,23:46:36,Wreck-It Ralph,Animation|Adventure|Comedy|Family|Fantasy,2012
1,41856,1772341,10,2013-02-28,18:18:32,Wreck-It Ralph,Animation|Adventure|Comedy|Family|Fantasy,2012
2,6582,1772341,7,2013-02-28,20:23:36,Wreck-It Ralph,Animation|Adventure|Comedy|Family|Fantasy,2012
3,40900,1772341,7,2013-02-28,19:35:22,Wreck-It Ralph,Animation|Adventure|Comedy|Family|Fantasy,2012
4,42790,1772341,8,2013-03-01,22:16:15,Wreck-It Ralph,Animation|Adventure|Comedy|Family|Fantasy,2012


In [32]:
df_final =pd.merge(df_2, use_file, on='user_id')
df_final = df_final.sort_values(by='date')
df_final.reset_index(drop=True, inplace=True)
df_final

,user_id,movie_id,rating,date,time,movie_title,genre,movie_year,twitter_id
0,29235,1772341,8,2013-02-28,23:46:36,Wreck-It Ralph,Animation|Adventure|Comedy|Family|Fantasy,2012,286577679
1,29420,381442,10,2013-02-28,21:45:25,Narco,Comedy,2004,148699617
2,6784,2091473,8,2013-02-28,21:52:36,Promised Land,Drama,2012,44841589
3,11902,1586265,5,2013-02-28,21:43:00,What to Expect When You're Expecting,Comedy|Drama|Romance,2012,57435432
4,21429,1024648,10,2013-02-28,18:18:22,Argo,Biography|Drama|Thriller,2012,173061672
5,3900,454876,8,2013-02-28,21:52:10,Life of Pi,Adventure|Drama|Fantasy,2012,72884681
6,38865,2400272,10,2013-02-28,20:58:48,Something Necessary,Drama,2013,90130884
7,7388,1428538,9,2013-02-28,21:05:19,Hansel & Gretel: Witch Hunters,Action|Fantasy|Horror,2013,486948701
8,13031,114787,9,2013-02-28,16:59:38,Underground,Comedy|Drama|War,1995,137260735
9,13031,101700,9,2013-02-28,17:02:00,Delicatessen,Comedy|Crime,1991,137260735


In [33]:
df_cross = pd.DataFrame(df_final.genre)

numeration = int(len(df_final))+1
df_cross['id'] = pd.Series(range(1,numeration))

In [34]:
df_cross

,genre,id
0,Animation|Adventure|Comedy|Family|Fantasy,1
1,Comedy,2
2,Drama,3
3,Comedy|Drama|Romance,4
4,Biography|Drama|Thriller,5
5,Adventure|Drama|Fantasy,6
6,Drama,7
7,Action|Fantasy|Horror,8
8,Comedy|Drama|War,9
9,Comedy|Crime,10


In [35]:
df_cross.isnull().values.any()

True

In [36]:
df_cross.isnull().sum()

genre    440
id         0
dtype: int64

In [37]:
df_cross['genre'] = df_cross.fillna('noGenre')

In [38]:
newdf_cross = pd.concat([pd.Series(row['id'], row['genre'].split('|'))
           for _, row in df_cross.iterrows()]).reset_index()

In [39]:
newdf_cross.columns

Index(['index', 0], dtype='object')

In [40]:
new_dfcross = pd.crosstab(newdf_cross[0], newdf_cross['index'])

In [41]:
new_dfcross

index,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,noGenre
0,,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
6,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [42]:
new_dfcross.rename(columns={'index': 'genre', 0: 'id'}, inplace=True)

In [43]:
new_dfcross.columns

Index(['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show',
       'Thriller', 'War', 'Western', 'noGenre'],
      dtype='object', name='index')

In [44]:
result = pd.merge(df_final.reset_index(),
                  new_dfcross.reset_index(), 
                  left_index=True, 
                  right_index=True)
result

,index,user_id,movie_id,rating,date,time,movie_title,genre,movie_year,twitter_id,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,noGenre
0,0,29235,1772341,8,2013-02-28,23:46:36,Wreck-It Ralph,Animation|Adventure|Comedy|Family|Fantasy,2012,286577679,...,0,0,0,0,0,0,0,0,0,0
1,1,29420,381442,10,2013-02-28,21:45:25,Narco,Comedy,2004,148699617,...,0,0,0,0,0,0,0,0,0,0
2,2,6784,2091473,8,2013-02-28,21:52:36,Promised Land,Drama,2012,44841589,...,0,0,0,0,0,0,0,0,0,0
3,3,11902,1586265,5,2013-02-28,21:43:00,What to Expect When You're Expecting,Comedy|Drama|Romance,2012,57435432,...,0,1,0,0,0,0,0,0,0,0
4,4,21429,1024648,10,2013-02-28,18:18:22,Argo,Biography|Drama|Thriller,2012,173061672,...,0,0,0,0,0,0,1,0,0,0
5,5,3900,454876,8,2013-02-28,21:52:10,Life of Pi,Adventure|Drama|Fantasy,2012,72884681,...,0,0,0,0,0,0,0,0,0,0
6,6,38865,2400272,10,2013-02-28,20:58:48,Something Necessary,Drama,2013,90130884,...,0,0,0,0,0,0,0,0,0,0
7,7,7388,1428538,9,2013-02-28,21:05:19,Hansel & Gretel: Witch Hunters,Action|Fantasy|Horror,2013,486948701,...,0,0,0,0,0,0,0,0,0,0
8,8,13031,114787,9,2013-02-28,16:59:38,Underground,Comedy|Drama|War,1995,137260735,...,0,0,0,0,0,0,0,1,0,0
9,9,13031,101700,9,2013-02-28,17:02:00,Delicatessen,Comedy|Crime,1991,137260735,...,0,0,0,0,0,0,0,0,0,0


In [45]:
result.columns

Index([      'index',     'user_id',    'movie_id',      'rating',
              'date',        'time', 'movie_title',       'genre',
        'movie_year',  'twitter_id',             0,      'Action',
             'Adult',   'Adventure',   'Animation',   'Biography',
            'Comedy',       'Crime', 'Documentary',       'Drama',
            'Family',     'Fantasy',   'Film-Noir',   'Game-Show',
           'History',      'Horror',       'Music',     'Musical',
           'Mystery',        'News',  'Reality-TV',     'Romance',
            'Sci-Fi',       'Short',       'Sport',   'Talk-Show',
          'Thriller',         'War',     'Western',     'noGenre'],
      dtype='object')

In [57]:
itm_file.movie_title.unique().shape

(33045,)

In [59]:
itm_file.duplicated('movie_title')

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
34399    False
34400    False
34401    False
34402    False
34403    False
34404    False
34405    False
34406    False
34407    False
34408    False
34409    False
34410    False
34411    False
34412    False
34413    False
34414    False
34415    False
34416    False
34417    False
34418    False
34419    False
34420    False
34421    False
34422    False
34423    False
34424    False
34425    False
34426    False
34427     True
34428    False
Length: 34429, dtype: bool


In [54]:
result.movie_title.unique().shape

(33045,)

In [50]:
col = ['Les Misérables','xx']
for i in col:
    print(result[result.movie_title == i].shape[0])

580
0


#  Pruebas

In [ ]:
df_testing = pd.DataFrame(df_final.genre)

pruebas = df_testing.head()
jj = int(pruebas.count()+1)
pruebas['id'] = pd.Series(range(1,jj))

In [ ]:
pruebas

In [ ]:
pd.concat([pd.Series(row['id'], row['genre'].split('|'))
           for _, row in pruebas.iterrows()]).reset_index()

In [ ]:
import pandas as pd
df = pd.DataFrame(['a b c']*100000, columns=['col'])
df.head()

In [ ]:
print (pd.DataFrame(dict(zip(range(3), [df['col'].apply(lambda x : x.split(' ')[i]) for i in range(3)]))).head())

In [ ]:
s = pruebas['genre'].str.split('|')
s

In [ ]:
q = pruebas['genre'].str.split('|').tolist()
q

In [ ]:
pd.DataFrame(dict(zip(range(16), [pruebas['genre'].apply(lambda x : x.split('|')[i]) for i in range(2)])))

In [ ]:
s = pruebas['genre'].str.split('|').apply(pd.Series, 1).stack()
s

In [ ]:
s.index = s.index.droplevel(-1)
s.name = 'genres'
s

In [ ]:
# s = pruebas['genre']
s = pruebas['genre'].str.split('|').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'genres'
pruebas.join(s.apply(lambda x: pd.Series(x.split('|'))))
pruebas = pruebas.join(s)
pruebas

In [ ]:
uni_gen = pruebas['genres'].unique()
uni_gen

In [ ]:
process = pd.get_dummies(pd.Series(list(pruebas['genres'])))

In [ ]:
result = pd.merge(pruebas.reset_index(),
                  process.reset_index(), 
                  left_index=True, 
                  right_index=True)
result

In [ ]:
new_df = pd.crosstab(result['id'], result['genres'])

In [ ]:
new_df

In [ ]:
data = pd.DataFrame(columns=(uni_gen))
data['id'] = pd.Series(range(1,jj))
data

In [ ]:
df_final.shape

In [ ]:
df_final['movie_title'].nunique()
#34314

In [ ]:
lista_m = itm_file['movie_title']

lista_f = df_final['movie_title']

In [ ]:
df_final[['dID','hID']].groupby(df['mID']).agg(['count', 'size', 'nunique'])

In [ ]:
lista_m.iloc['Joker']

is_male = lista_m.loc[:,] == 'Joker'
df_male = lista_m.loc[is_male]
df_male.head()

In [ ]:
lista_f = pd.DataFrame(lista_f)
lista_f

In [ ]:
#surveys_df.loc[[0, 10], :] surveys_df[surveys_df.year == 2002]

lista_m = pd.DataFrame(lista_m)
lista_m['movie_title'].iloc[2]

In [ ]:
lista_m.movie_title.dtypes

In [ ]:
lista_m.movie_title =='The Oxford and Cambridge University Boat Race'

In [ ]:
is_joker = lista_m.loc[:, 'movie_title'] == 'Upstarts'
df_joker = lista_m.loc[is_joker]
df_joker.head()

In [ ]:
is_joker.unique()

In [ ]:
comparacion = []
for item in lista_m:
    if item in lista_m:
        comparacion.append(item)

if len(comparacion) > 0:
    print('Ambas listas contienen estos elementos')
    for item in comparacion: print('%s' % item)

else:
    print('No existe ningun elemento igual en las listas')

In [ ]:
lista1=["paco","pepe","luis"]

lista2=["diego","mari","luis"]

comparacion = []

 

for item in lista1:

  if item in lista2:

    comparacion.append(item)

 

if len(comparacion) > 0:

  print 'Ambas listas contienen estos elementos'

  for item in comparacion: print '%s' % item

else:

  print 'No existe ningun elemento igual en las listas'

In [ ]:
n_col = df_final.columns

for col in n_col:
    print(df[col].unique)